## Import PuLP package

In [3]:
import pulp
import pandas as pd

In [2]:
# pulp.pulpTestAll()

## Define Constant

ある工場ではp1, p2, p3, p4を製造している。
製品p1,p2,p3,p4には、原料m1,m2,m3の3種類が必要。
必要するは、require.csvのrequire列に記載。

- requires.csv   :各製品において必要な原料の量
- stocks.csv     :原料の在庫量
- gains.csv     :製品を生産した際の利得

In [7]:
require_df = pd.read_csv('requires.csv')
stock_df = pd.read_csv('stocks.csv')
gain_df = pd.read_csv('gains.csv')

In [10]:
products = gain_df['p'].to_list()
materials = stock_df['m'].to_list()

In [20]:
stock = { row.m:row.stock for row in stock_df.itertuples()}
stock
# dict(zip(stock_df['m'], stock_df['stock']))

{'m1': 35, 'm2': 22, 'm3': 27}

In [21]:
gain = { row.p:row.gain for row in gain_df.itertuples()}
gain

{'p1': 3, 'p2': 4, 'p3': 4, 'p4': 5}

In [22]:
require = {(row.p, row.m):row.require for row in require_df.itertuples()}
require

{('p1', 'm1'): 2,
 ('p1', 'm2'): 0,
 ('p1', 'm3'): 1,
 ('p2', 'm1'): 3,
 ('p2', 'm2'): 2,
 ('p2', 'm3'): 0,
 ('p3', 'm1'): 0,
 ('p3', 'm2'): 2,
 ('p3', 'm3'): 2,
 ('p4', 'm1'): 2,
 ('p4', 'm2'): 2,
 ('p4', 'm3'): 2}

## Create Variable

変数x_pは製品pの生産量。

In [32]:
# 線形計画問題の場合 Continuous
# 整数計画問題の場合 Integer
x = pulp.LpVariable.dicts('x', products, cat='Integer')
x
# x = {}
# for p in products:
#     x[p] = pulp.LpVariable(f'x_{p}', cat='Continuous')

{'p1': x_p1, 'p2': x_p2, 'p3': x_p3, 'p4': x_p4}

## Create Model

- `LpProblem('name', sense=LpMinimize or LpMaximize)

In [33]:
prob = pulp.LpProblem('LP2', pulp.LpMaximize)

## Create Constraint

In [34]:
# すべての変数が0以上
for p in products:
    prob += x[p] >= 0

# 生産に使用した原料が在庫以下
for m in materials:
    prob += pulp.lpSum([require[p,m] * x[p] for p in products]) <= stock[m]

## Create Target function

In [35]:
prob += pulp.lpSum([gain[p] * x[p] for p in products])

## Exec Solver

- Default solver is CBC. You can use GLPK if installed.

In [36]:
status = prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/norihito/workspace/optimization-problems/.venv/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/01b8bdae75a845ae9a9b8982a9cfa2b9-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/01b8bdae75a845ae9a9b8982a9cfa2b9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 38 RHS
At line 46 BOUNDS
At line 51 ENDATA
Problem MODEL has 7 rows, 4 columns and 13 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 80.4286 - 0.00 seconds
Cgl0004I processed model has 3 rows, 4 columns (4 integer (0 of which binary)) and 9 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of -76 found by DiveCoefficient after 0 iterations and 0 nodes (0.03 seconds)
Cbc0038I Full problem 3 rows 4 columns, reduced to 3 rows 3 columns


## Check status

- You can check the status by `LpStatus` which shows 
    {0: 'Not Solved', 1: 'Optimal', -1:'Infeasible', -2: 'Unbounded', -3: 'Undefined'}

In [37]:
pulp.LpStatus

{0: 'Not Solved',
 1: 'Optimal',
 -1: 'Infeasible',
 -2: 'Unbounded',
 -3: 'Undefined'}

In [38]:
pulp.LpStatus[prob.status]
# pulp.LpStatus[status]

'Optimal'

## Result 

In [39]:
for p in products:
    print(p, '=', x[p].value())
print('obj=', prob.objective.value())

p1 = 13.0
p2 = 3.0
p3 = 7.0
p4 = -0.0
obj= 79.0
